In [ ]:
#1-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_1.csv')
pd.set_option('display.max_columns', None)
print(df.head())

   year  gender  지역코드    금액1    금액2
0  2018       0   143  70713  43210
1  2019       0   183  40306  39663
2  2017       0   129  26646  41631
3  2019       0   161  56843  12455
4  2019       0   174  26371   9014


In [ ]:
df['총대출액'] = df['금액1'] + df['금액2']
dfgender = df.groupby(['year','gender','지역코드'])['총대출액'].sum().reset_index()
df_pivot = dfgender.pivot_table(index=['year', '지역코드'], columns='gender', values='총대출액', fill_value=0)
df_pivot['abs_diff'] = abs(df_pivot[0]-df_pivot[1])
print(df_pivot['abs_diff'].idxmax())
# 143이 가장 큼

(np.int64(2018), np.int64(143))


In [ ]:
#1-2
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_2.csv')
print(df.head())

     연도    구분  범죄유형1  범죄유형2  범죄유형3  범죄유형4  범죄유형5  범죄유형6  범죄유형7  범죄유형8  범죄유형9  \
0  2000  발생건수    302    635    470    306    271    900    220    814    321   
1  2000  검거건수    302    430    470    187    271    199    220    230    321   
2  2001  발생건수    969    543    691    613    585    391    476    360    659   
3  2001  검거건수    121    352    691    613    574    158    476    360    575   
4  2002  발생건수    982    389    886    762    766    443    704    330    684   

   범죄유형10  
0     666  
1     408  
2     513  
3     513  
4     846  


In [ ]:
df1 = df[df['구분'] == '검거건수'].set_index('연도').drop(columns='구분')
df2 = df[df['구분'] == '발생건수'].set_index('연도').drop(columns='구분')
dfratio = df1 / df2
dfmax = dfratio.idxmax(axis=1)    # 각 연도별 최대 검거율을 가진 범죄유형
dfmax_reset = dfmax.reset_index().rename({0:'범죄유형'}, axis=1)
df1_melt = df1.reset_index().melt(id_vars='연도', var_name = '범죄유형', value_name = '검거건수')
dffinal = pd.merge(dfmax_reset, df1_melt, on = ['연도', '범죄유형'], how = 'left')
total_sum = dffinal['검거건수'].sum()
print('총 검거건수 합계 = ', total_sum)

KeyError: '구분'

In [ ]:
#1-3
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_1_3.csv')
print(df.head())

          부서 등급     평균만족도  근속연수  교육참가횟수
0    Finance  C  4.023911  17.0     8.0
1  Marketing  A  3.756698  14.0    16.0
2         IT  B  2.628697  21.0    13.0
3  Marketing  A  3.196616   NaN    18.0
4  Marketing  C  4.770665  28.0    10.0


In [ ]:
# 결측치를 확인합니다.
print(df.isna().sum())

부서         0
등급         0
평균만족도     10
근속연수      10
교육참가횟수     0
dtype: int64


In [ ]:
# 평균만족도 칼럼은 해당 칼럼의 평균으로 대치합니다.
df['평균만족도'] = df['평균만족도'].fillna(df['평균만족도'].mean())

In [ ]:
# 부서와 등급에 따라 평균근속연수로 대치합니다.
import numpy as np
mean_tenure = (
    df.groupby(['부서', '등급'])['근속연수']
    .mean()
    .apply(np.floor)
    .reset_index()
    .rename(columns = {"근속연수":"평균근속연수"})
    )
# merge로 결측치를 대치하기
df = df.merge(mean_tenure, on = ['부서', '등급'], how = 'left')
df['근속연수'] = np.where(
    df['근속연수'].isnull(),  # 조건 : 근속연수가 결측치인 경우
    df['평균근속연수'],
    df['근속연수']
)
df.drop(columns = ['평균근속연수'], inplace = True)

In [ ]:
# 결측치가 처리된 결과를 확인합니다.
print(df.isna().sum())

부서        0
등급        0
평균만족도     0
근속연수      0
교육참가횟수    0
dtype: int64


In [ ]:
# 부서가 'HR'이고 등급이 'A'인 사람들의 평균 근속연수를 계산합니다.
A = df.loc[(df['부서'] == 'HR') & (df['등급'] == 'A'), '근속연수'].mean()

# 부서가 'Sales'이고 등급이 'B'인 사람들의 평균 교육참가횟수를 계산하시오.
B = df.loc[(df['부서'] == 'Sales') & (df['등급'] == 'B'), '교육참가횟수'].mean()

In [ ]:
# 두 조건의 합을 구합니다.
result = A+B
print(result)

25.225


In [ ]:
#2
import pandas as pd
import numpy as np
# 데이터 로드
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_2_test.csv')

In [ ]:
# 데이터 탐색 : 모델을 적합하기 전 데이터에 결측치가 있는지 확인해야 합니다. 결측치가 존재할 경우 모델 적합 시 에러가 발생할 수 있습니다.
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      1680 non-null   int64 
 1   지역      1680 non-null   object
 2   등급      1680 non-null   object
 3   농업면적    1680 non-null   int64 
 4   연도      1680 non-null   int64 
 5   라벨      1680 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 78.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      720 non-null    int64 
 1   지역      720 non-null    object
 2   등급      720 non-null    object
 3   농업면적    720 non-null    int64 
 4   연도      720 non-null    int64 
 5   라벨      720 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 33.9+ KB
None


In [ ]:
# 테스트 데이터를 활용한 최종 결과를 제출하기 위해 ID 칼럼을 따로 저장합니다.
## 2025년 6월 체험판에서는 pred열만을 제출하는 것을 권장했으므로 ID 칼럼 따로 저장하는 절차 필요가 없음.
test_id = test['ID']

# 독립변수와 종속변수 지정
train_X = train.drop(['라벨'], axis=1)
train_y = train['라벨']
test_X = test.drop(['라벨'], axis=1)
test_y = test['라벨']

In [ ]:
# 데이터 분할 : 모델 성능 확인을 위해 훈련 데이터의 일부를 검증 데이터로 나누겠습니다.
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=1)

In [ ]:
#4 범주-수치 칼럼 분할(원-핫 인코딩 전단계)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

In [ ]:
#5 원-핫 불러오기
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

In [ ]:
#6 원-핫 전처리
train_X_preprocessed = onehotencoder.fit_transform(train_X[cat_columns])
valid_X_preprocessed = onehotencoder.transform(valid_X[cat_columns])
test_X_preprocessed = onehotencoder.transform(test_X[cat_columns])

In [ ]:
#7 랜포 적합
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X_preprocessed, train_y)

RandomForestClassifier(random_state=1)

In [ ]:
#8 검증 - 성능 확인
from sklearn.metrics import f1_score
pred_val = rf.predict(valid_X_preprocessed)
print(f1_score(valid_y, pred_val, average = 'macro'))

0.31437125748503


In [ ]:
#9 최종예측

test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns = ['pred'])

In [ ]:
#10 생성, 저장
result = pd.concat([test_id, test_pred], axis = 1)
print(result.head(2))

result.to_csv('result.csv', index = False)

     ID  pred
0  1094     0
1  1341     0


In [ ]:
train_X_full = np.concatenate([train_X_preprocessed, valid_X_preprocessed], axis=0)
train_y_full = np.concatenate([train_y, valid_y], axis=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth' : [10,20,30],
              'min_samples_split' : [2,5,10]}

rf_search = GridSearchCV(estimator = rf,
                         param_grid = param_grid,
                         cv = 3,
                         scoring = 'f1_macro')

rf_search.fit(train_X_full, train_y_full)
print(rf_search.best_score_)

test_pred2 = rf_search.predict(test_X_preprocessed)
test_pred2 = pd.DataFrame(test_pred2, columns = ['pred'])

result = pd.concat([test_id, test_pred2], axis=1)
result.to_csv('result2.csv', index=False)

0.3115942028985507


In [ ]:
#3-1
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_1.csv')

In [ ]:
#3-1-1 (유의하지 않은 설명변수의 개수 구하기)
train = df[df['id'] <= 140]
test = df[df['id'] > 140]
# 다중회귀모델 적합
from statsmodels.formula.api import ols
formula = 'design ~ tenure + f2 + f3 + f4 + f5'
model = ols(formula, data = train).fit()

p_values = model.pvalues

non_significant_vars = (p_values[1:] >= 0.05).sum()
print(non_significant_vars)

2


In [ ]:
#3-1-2 (훈련데이터의 예측값과 실제값의 상관계수 구하기)

y_pred = model.predict(train)   #예측값
y_real = train['design']        #실제값

from scipy.stats import pearsonr
correlation, _ = pearsonr(y_pred, y_real)
print(correlation)

# 또는 #

y_pred_series = pd.Series(y_pred)
y_real_series = pd.Series(y_real)

correlation2 = y_real_series.corr(y_pred_series)
print(correlation2)

0.914750154303963
0.9147501543039637


In [ ]:
# 3-1-3 (테스트 데이터 RMSE 구하기)

y_pred_test = model.predict(test)
from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_pred_test, test['design'])
print(rmse)

4.396152958589427


In [ ]:
# 3-2
import numpy as np
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/9_3_2.csv')
print(df.head())

        col1       col2  Phone_Service  Tech_Insurance  churn
0  54.967142  34.630888              0               1      0
1  48.617357  39.547083              0               0      0
2  56.476885  23.007162              1               0      1
3  65.230299  32.814846              1               0      1
4  47.658466  26.746787              0               1      1


In [ ]:
# 3-2-1 (로지스틱 회귀 적합 후 p-value 구하기)

from statsmodels.formula.api import logit
formula = 'churn ~ col1 + col2 + Phone_Service + Tech_Insurance'
model = logit(formula, data=df).fit()
col1_pvalue = round(model.pvalues['col1'], 3)
print(col1_pvalue)

Optimization terminated successfully.
         Current function value: 0.640721
         Iterations 5
0.0


In [ ]:
# 3-2-2 (이탈 확률 오즈비 구하기 (폰서비스))

odds_ratio_phone_service = round(np.exp(model.params['Phone_Service']), 3)
print(odds_ratio_phone_service)

1.867


In [ ]:
# 3-2-3 (적합된 모델의 예측 확률이 0.3 이상인 고객의 수 계산)

predicted_probs = model.predict(df)
num_customer_above_03 = sum(predicted_probs > 0.3)
print(num_customer_above_03)

450
